## Import

In [ ]:
import pandas as pd
import numpy as np

## Helper Functions

In [ ]:
# formats the column names in snakecase style
def snakecase_cols(df):

    new_cols = ["_".join(col.lower().split(" ")) for col in df.columns]
    df.columns = new_cols
    return df

# removes rows where the data have been redacted
def remove_redacted(df, columns):

    non_redacted_mask = None

    for col in columns:
        if non_redacted_mask is None:
            non_redacted_mask = ~(df[col] == "s")
        else:
            non_redacted_mask = non_redacted_mask & ~(df[col] == "s")

    return df[non_redacted_mask]

# processes the provided df into desired format and removes unneeded data
def process_data(df, sheet=None):

    # turn column names to snakecase
    df = snakecase_cols(df)

    # drop the unneeded columns
    df = df.drop(columns=["mean_scale_score", "number_tested"])

    # keep only desired categories
    if sheet == "SWD":
      df = df[df["category"] == "SWD"]
    elif sheet == "ELL":
      df = df[df["category"] == "Current ELL"]

    # drop the rows showing All Grades
    df = df[df["grade"] != "All Grades"]

    # remove redacted rows
    df = remove_redacted(df, ["#_level_1", "%_level_1", "#_level_2", "%_level_2",
                              "#_level_3", "%_level_3", "#_level_4", "%_level_4", "#_level_3+4", "%_level_3+4"])

    # change the numerical columns to proper data types
    df = df.astype({"#_level_1" : np.int32, "%_level_1" : np.float32,
                    "#_level_2" : np.int32, "%_level_2" : np.float32, "#_level_3" : np.int32,
                    "%_level_3" : np.float32, "#_level_4" : np.int32, "%_level_4" : np.float32,
                    "#_level_3+4" : np.int32, "%_level_3+4" : np.float32
                    })

    # rename the columns
    df = df.rename(columns={"#_level_1" : "num_level_1",
                            "%_level_1" : "percent_level_1",
                            "#_level_2" : "num_level_2",
                            "%_level_2" : "percent_level_2",
                            "#_level_3" : "num_level_3",
                            "%_level_3" : "percent_level_3",
                            "#_level_4" : "num_level_4",
                            "%_level_4" : "percent_level_4",
                            "#_level_3+4" : "num_proficient",
                            "%_level_3+4" : "percent_proficient"
                            })

    # turn percentages to decimals
    for col in df.columns:
      if "percent" in col:
        df[col] = df[col].apply(lambda value : value / 100)

    # keep only the data from 2023 and 2024
    df = df[(df["year"] == 2023) | (df["year"] == 2024)]

    return df

## Borough

Process and reformat the performance data organized by borough.

### ELA

In [ ]:
boro_ela_all = pd.read_excel("FILEPATH", sheet_name="ELA - All")
boro_ela_swd = pd.read_excel("FILEPATH", sheet_name="ELA - SWD")
boro_ela_ell = pd.read_excel("FILEPATH", sheet_name="ELA - ELL")

boro_ela_all = process_data(boro_ela_all)
boro_ela_swd = process_data(boro_ela_swd, sheet="SWD")
boro_ela_ell = process_data(boro_ela_ell, sheet="ELL")

boro_ela = pd.concat([boro_ela_all, boro_ela_swd, boro_ela_ell], ignore_index=True)

### Math

In [ ]:
boro_math_all = pd.read_excel("FILEPATH", sheet_name="Math - All")
boro_math_swd = pd.read_excel("FILEPATH", sheet_name="Math - SWD")
boro_math_ell = pd.read_excel("FILEPATH", sheet_name="Math - ELL")

boro_math_all = process_data(boro_math_all)
boro_math_swd = process_data(boro_math_swd, sheet="SWD")
boro_math_ell = process_data(boro_math_ell, sheet="ELL")

boro_math = pd.concat([boro_math_all, boro_math_swd, boro_math_ell], ignore_index=True)

### ELA and Math

In [ ]:
boro_ela["subject"] = ["ELA" for i in range(boro_ela.shape[0])]
boro_math["subject"] = ["Math" for i in range(boro_math.shape[0])]

boro_df = pd.concat([boro_ela, boro_math], ignore_index=True)
boro_df = boro_df.sort_values(by=["borough", "subject", "category", "year", "grade"], ignore_index=True)

In [ ]:
# reorganize the df to have the percents and counts of each level as rows instead of columns
new_format_dict = {
    "borough" : [],
    "year" : [],
    "subject" : [],
    "category" : [],
    "grade" : [],
    "level" : [],
    "num_level" : [],
    "percent_level" : [],
}

for i in boro_df.index:
  boro = boro_df.loc[i, "borough"]
  year = boro_df.loc[i, "year"]
  subject = boro_df.loc[i, "subject"]
  category = boro_df.loc[i, "category"]
  grade = boro_df.loc[i, "grade"]

  for lvl in ["level_1", "level_2", "level_3", "level_4", "proficient"]:

    new_format_dict["borough"].append(boro)
    new_format_dict["year"].append(year)
    new_format_dict["subject"].append(subject)
    new_format_dict["category"].append(category)
    new_format_dict["level"].append(" ".join(lvl.capitalize().split("_")))
    new_format_dict["grade"].append(grade)

    for metric in ["num", "percent"]:

      col = f"{metric}_{lvl}"
      value = boro_df.loc[i, col]

      if "num" in col:
        new_format_dict["num_level"].append(value)
      else:
        new_format_dict["percent_level"].append(value)

boro_df = pd.DataFrame(new_format_dict)
boro_df["id"] = [i for i in range(boro_df.shape[0])]

In [ ]:
boro_df.head(10)

## District

Process and reformat the performance data organized by school district.

### ELA

In [ ]:
district_ela_all = pd.read_excel("FILEPATH", sheet_name="ELA - All")
district_ela_swd = pd.read_excel("FILEPATH", sheet_name="ELA - SWD")
district_ela_ell = pd.read_excel("FILEPATH", sheet_name="ELA - ELL")

district_ela_all = process_data(district_ela_all)
district_ela_swd = process_data(district_ela_swd, sheet="SWD")
district_ela_ell = process_data(district_ela_ell, sheet="ELL")

district_ela = pd.concat([district_ela_all, district_ela_swd, district_ela_ell], ignore_index=True)

### Math

In [ ]:
district_math_all = pd.read_excel("FILEPATH", sheet_name="Math - All")
district_math_swd = pd.read_excel("FILEPATH", sheet_name="Math - SWD")
district_math_ell = pd.read_excel("FILEPATH", sheet_name="Math - ELL")

district_math_all = process_data(district_math_all)
district_math_swd = process_data(district_math_swd, sheet="SWD")
district_math_ell = process_data(district_math_ell, sheet="ELL")

district_math = pd.concat([district_math_all, district_math_swd, district_math_ell], ignore_index=True)

### ELA and Math

In [ ]:
district_ela["subject"] = ["ELA" for i in range(district_ela.shape[0])]
district_math["subject"] = ["Math" for i in range(district_math.shape[0])]

districts_df = pd.concat([district_ela, district_math], ignore_index=True)
districts_df = districts_df.sort_values(by=["district", "subject", "category", "year", "grade"], ignore_index=True)

In [ ]:
districts_df.head()

In [ ]:
# reorganize the df to have the percents and counts of each level as rows instead of columns
new_format_dict = {
    "district" : [],
    "year" : [],
    "subject" : [],
    "category" : [],
    "grade" : [],
    "level" : [],
    "num_level" : [],
    "percent_level" : []
}

for i in districts_df.index:
  district = districts_df.loc[i, "district"]
  year = districts_df.loc[i, "year"]
  subject = districts_df.loc[i, "subject"]
  category = districts_df.loc[i, "category"]
  grade = districts_df.loc[i, "grade"]

  for lvl in ["level_1", "level_2", "level_3", "level_4", "proficient"]:

    new_format_dict["district"].append(district)
    new_format_dict["year"].append(year)
    new_format_dict["subject"].append(subject)
    new_format_dict["category"].append(category)
    new_format_dict["level"].append(" ".join(lvl.capitalize().split("_")))
    new_format_dict["grade"].append(grade)

    for metric in ["num", "percent"]:

      col = f"{metric}_{lvl}"
      value = districts_df.loc[i, col]

      if "num" in col:
        new_format_dict["num_level"].append(value)
      else:
        new_format_dict["percent_level"].append(value)

districts_df = pd.DataFrame(new_format_dict)
districts_df["id"] = [i for i in range(districts_df.shape[0])]

In [ ]:
districts_df.head(10)

## School-Based

Process and reformat the performance data organized by school.

### ELA

#### Non-Charter Schools

In [ ]:
non_charter_ela_all = pd.read_excel("FILEPATH", sheet_name="ELA - All")
non_charter_ela_swd = pd.read_excel("FILEPATH", sheet_name="ELA - SWD")
non_charter_ela_ell = pd.read_excel("FILEPATH", sheet_name="ELA - ELL")

non_charter_ela_all = process_data(non_charter_ela_all)
non_charter_ela_swd = process_data(non_charter_ela_swd, sheet="SWD")
non_charter_ela_ell = process_data(non_charter_ela_ell, sheet="ELL")

#### Charter Schools

In [ ]:
charter_ela_all = pd.read_excel("FILEPATH", sheet_name="ELA")
charter_ela_all = process_data(charter_ela_all)

#### Concat Together

In [ ]:
schools_ela = pd.concat([non_charter_ela_all, non_charter_ela_swd, non_charter_ela_ell, charter_ela_all], ignore_index=True)

### Math

#### Non-Charter Schools

In [ ]:
non_charter_math_all = pd.read_excel("FILEPATH", sheet_name="Math - All")
non_charter_math_swd = pd.read_excel("FILEPATH", sheet_name="Math - SWD")
non_charter_math_ell = pd.read_excel("FILEPATH", sheet_name="Math - ELL")

non_charter_math_all = process_data(non_charter_math_all)
non_charter_math_swd = process_data(non_charter_math_swd, sheet="SWD")
non_charter_math_ell = process_data(non_charter_math_ell, sheet="ELL")

#### Charter Schools

In [ ]:
charter_math_all = pd.read_excel("FILEPATH", sheet_name="Math")
charter_math_all = process_data(charter_math_all)

#### Concat Together

In [ ]:
schools_math = pd.concat([non_charter_math_all, non_charter_math_swd, non_charter_math_ell, charter_math_all], ignore_index=True)

### ELA and Math

In [ ]:
schools_ela["subject"] = ["ELA" for i in range(schools_ela.shape[0])]
schools_math["subject"] = ["Math" for i in range(schools_math.shape[0])]

In [ ]:
schools_df = pd.concat([schools_ela, schools_math], ignore_index=True)
schools_df["district"] = schools_df["dbn"].apply(lambda dbn : int(dbn[:2]))
schools_df = schools_df.sort_values(by=["district", "dbn", "school_name", "subject", "category", "year", "grade"], ignore_index=True)

In [ ]:
schools_df.head()

### Difference from Borough

Calculate the differences between school proficiency rate and borough proficiency rate.

In [ ]:
diff_from_boro = []
no_boro_data_set = set()

boro_codes = {"M" : "MANHATTAN", "R" : "STATEN ISLAND", "K" : "BROOKLYN", "X" : "BRONX", "Q" : "QUEENS"}

boro_subset = boro_df[boro_df["level"] == "Proficient"]

for i in schools_df.index:

    school_proficient_rate = schools_df.loc[i, "percent_proficient"]
    cat = schools_df.loc[i, "category"]
    year = schools_df.loc[i, "year"]
    subject = schools_df.loc[i, "subject"]
    grade = schools_df.loc[i, "grade"]
    boro = boro_codes[schools_df.loc[i, "dbn"][2]]

    try:
        boro_proficient_rate = boro_subset[(boro_subset["category"] == cat) & (boro_subset["year"] == year) & (boro_subset["borough"] == boro) & (boro_subset["subject"] == subject) & (boro_subset["grade"] == grade)]["percent_level"].values[0]
        diff_from_boro.append(school_proficient_rate - boro_proficient_rate)
    except IndexError:
        no_boro_data_set.add((year, cat, boro, subject, grade))
        diff_from_boro.append(np.nan)

schools_df["diff_from_boro_percent_proficient"] = diff_from_boro

In [ ]:
no_boro_data_set

### Difference from District

Calculate the differences between school proficiency rate and school district proficiency rate.

In [ ]:
diff_from_district = []
no_district_data_set = set()

district_subset = districts_df[districts_df["level"] == "Proficient"]

for i in schools_df.index:

    school_proficient_rate = schools_df.loc[i, "percent_proficient"]
    cat = schools_df.loc[i, "category"]
    year = schools_df.loc[i, "year"]
    district = schools_df.loc[i, "district"]
    subject = schools_df.loc[i, "subject"]
    grade = schools_df.loc[i, "grade"]

    try:
        district_proficient_rate = district_subset[(district_subset["category"] == cat) & (district_subset["year"] == year) & (district_subset["district"] == district) & (district_subset["subject"] == subject) & (district_subset["grade"] == grade)]["percent_level"].values[0]
        diff_from_district.append(school_proficient_rate - district_proficient_rate)
    except IndexError:
        no_district_data_set.add((year, cat, district, subject, grade))
        diff_from_district.append(np.nan)

schools_df["diff_from_district_percent_proficient"] = diff_from_district

In [ ]:
no_district_data_set

In [ ]:
schools_df.head()

In [ ]:
# reformat the df so that num of levels and percent of levels are organized as rows
new_format_dict = {
    "district" : [],
    "dbn" : [],
    "school_name" : [],
    "year" : [],
    "subject" : [],
    "category" : [],
    "grade" : [],
    "level" : [],
    "num_level" : [],
    "percent_level" : [],
    "diff_from_district_percent_proficient" : [],
    "diff_from_boro_percent_proficient" : []
    }

for i in schools_df.index:
  district = schools_df.loc[i, "district"]
  dbn = schools_df.loc[i, "dbn"]
  school_name = schools_df.loc[i, "school_name"]
  year = schools_df.loc[i, "year"]
  subject = schools_df.loc[i, "subject"]
  category = schools_df.loc[i, "category"]
  grade = schools_df.loc[i, "grade"]
  diff_from_district_percent_proficient = schools_df.loc[i, "diff_from_district_percent_proficient"]
  diff_from_boro_percent_proficient = schools_df.loc[i, "diff_from_boro_percent_proficient"]

  for lvl in ["level_1", "level_2", "level_3", "level_4", "proficient"]:

    new_format_dict["district"].append(district)
    new_format_dict["dbn"].append(dbn)
    new_format_dict["school_name"].append(school_name)
    new_format_dict["year"].append(year)
    new_format_dict["subject"].append(subject)
    new_format_dict["category"].append(category)
    new_format_dict["grade"].append(grade)
    new_format_dict["diff_from_district_percent_proficient"].append(diff_from_district_percent_proficient)
    new_format_dict["diff_from_boro_percent_proficient"].append(diff_from_boro_percent_proficient)
    new_format_dict["level"].append(" ".join(lvl.capitalize().split("_")))

    for metric in ["num", "percent"]:

      col = f"{metric}_{lvl}"
      value = schools_df.loc[i, col]

      if "num" in col:
        new_format_dict["num_level"].append(value)
      else:
        new_format_dict["percent_level"].append(value)

schools_df = pd.DataFrame(new_format_dict)

In [ ]:
schools_df["id"] = [i for i in range(schools_df.shape[0])]

In [ ]:
schools_df.head(10)

## Export

In [ ]:
schools_df.to_csv("FILEPATH", index=False)